In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth',200)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt

import sys
%load_ext autoreload
%autoreload 2
sys.path.append("../src")
from regression_lgbmodel import LgbRegression
from feature_engineering import feat_engineering, city_preprocess, data_day_group, data_merge


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
df_power = pd.read_csv('../../../初赛1008/训练集/power.csv')
df_his = pd.read_csv('../../../初赛1008/训练集/power_forecast_history.csv')
df_stub = pd.read_csv('../../../初赛1008/训练集/stub_info.csv')

df_test_his = pd.read_csv('../../../初赛1008/测试集/power_forecast_history.csv')
df_test_stub = pd.read_csv('../../../初赛1008/测试集/stub_info.csv')
df_stub.equals(df_test_stub)
df_his_all = pd.concat([df_his, df_test_his]).reset_index(drop=True)
df_his_all.head()

True

,id_encode,hour,ele_price,ser_price,after_ser_price,total_price,f1,f2,f3,ds
0,0,0.0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415
1,0,1.0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415
2,0,2.0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415
3,0,3.0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415
4,0,4.0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415


In [18]:

# df_city['province'].value_counts()
# df_city[df_city['province']=='上海市']
# df_stub[df_stub['h3']=='85309d6ffffffff']

## 特征工程

In [19]:
df_city = pd.read_csv('../../../初赛1008/h3_feature.csv',encoding='GBK')
df_weather = pd.read_parquet('../../../初赛1008/city_weather.parquet')
# df_city = city_preprocess(df_city, df_weather)
# df_city

In [21]:
df_his_day, df_power_day, df_power_mean = data_day_group(df_his_all, df_power)
df = data_merge(df_stub, df_his_day, df_power_day, df_power_mean, df_city)
df = feat_engineering(df)

df = df.merge(df_city, on=['h3', 'time'], how='left')
# 删除填补的空缺字段
df = df_his_day[['id_encode','ds']].merge(df, on=['id_encode','ds'], how='left')
df.shape


df_train = df[df['ds']<20230415]
df_test = df[df['ds']>=20230415]
# 删除填补的空缺值数据
df_train = df_train[(~df_train['power_sum'].isnull())]
# 删除部分20230101空数据
df_train = df_train[df_train[['ele_price_mean', 'ser_price_mean']].isnull().sum(axis=1) == 0]
# 时间截断
df_train = df_train[~df_train['power_1week'].isnull()]
# df_train = df_train[df_train['ds']>20220515]

# 掉分了
# df_h3 = df_train.groupby(['id_encode','h3'])['power_sum'].mean().reset_index().rename(columns={'power_sum':'h3_target_encode'})
# df_train = df_train.merge(df_h3, on=['id_encode','h3'], how='left')
# df_test = df_test.merge(df_h3, on=['id_encode','h3'], how='left')

# df_power_tmp = df_train.groupby(['id_encode']).agg({'power_sum':['mean','std']}).reset_index()
# df_power_tmp.columns = ['id_encode', 'power_TE', 'power_std_TE']
# df_train = df_train.merge(df_power_tmp, on=['id_encode'], how='left')
# df_test = df_test.merge(df_power_tmp, on=['id_encode'], how='left')
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape
# df_train.to_parquet(f'../data/interim/df_train_{oof_score}_A225.7.parquet')

df_time_all.shape: (186000, 4)
ori df: (186000, 29)


TypeError: incompatible index of inserted column with frame index

## model

In [ ]:
feats = [x for x in df_train.columns if x not in ['ds',  'time', 'month_num', 'year','label','power',
                                            'power_sum','power_std','pred','pred_r','quarter','h3',
                                            'all_price','折扣','city', 'Holiday','Holiday Length', 'weather',
                                            'weather_status']]#  #'Holiday Length',
feats
len(feats)

In [ ]:
# df_train1 = df_train[df_train['label']>0]
# df_train1 = df_train[df_train['is_holiday']==0]
# df_train['is_holiday'].value_counts()
lgbmodel = LgbRegression(feats)
df_train_pred, fold_importance_df, oof_score = lgbmodel.train(df_train, label='label', Stratifiedcol='id_encode')

In [ ]:
# model_all = lgbmodel.train_all(df_train, label='power_sum', 
#                                 best_iter=4076)

In [ ]:
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# # 后处理
# df_train_pred['pred_r'] = df_train_pred['pred'].apply(lambda x: 0 if x < 0 else x)
# np.sqrt(mean_squared_error(df_train_pred['label'] * (df_train_pred['power'] + 1), 
#                            df_train_pred['pred'] * (df_train_pred['power'] + 1)))
# np.sqrt(mean_squared_error(df_train_pred['power_sum'], df_train_pred['pred_r']))

current_time = datetime.datetime.now()
current_time_str = current_time.strftime('%Y%m%d_%H%M')
save_path = f'../data/interim/oof_{current_time_str}_oofrmse{oof_score}.csv'
df_train_pred[['id_encode', 'ds', 'power_sum', 'pred']].to_csv(save_path)
save_path

In [ ]:
df_oof_rmse = df_train_pred.groupby('id_encode').apply(lambda x: np.sqrt(mean_squared_error(x['power_sum'], x['pred'])))
df_oof_rmse.sort_values(0).tail(10)

In [ ]:
fold_importance_df.head(50)

In [ ]:
cid = 197
fig = plt.figure(figsize=(20, 4))
df_tmp = df_train_pred[(df_train_pred['id_encode']==cid)]
plt.plot(df_tmp['time'], df_tmp['label'],c='b')
plt.title(f'id_encode {cid}')
plt.plot(df_tmp['time'],df_tmp['pred'],c='r');

In [ ]:
def plot_sub_df(cids, df, pred_flag=0):
    col_num = 3
    fig, axes = plt.subplots(3, col_num, figsize=(20, 9), sharey=True)
    for i,cid in tqdm(enumerate(cids)):
        ax = axes[i // col_num, i % col_num]
        df_tmp = df[(df['id_encode']==cid)]
        ax.plot(df_tmp['time'], df_tmp['power_sum'],c='b')
        rmse = int(df_oof_rmse[cid])
        ax.set_title(f'id_encode {cid}_rmse{rmse}')
        if pred_flag == 1:
            ax.plot(df_tmp['time'],df_tmp['pred'],c='r')
    plt.tight_layout()
cids = np.arange(9) + 40
plot_sub_df(cids, df_train_pred, pred_flag=1)

## 提交

In [ ]:
# df_test['pred'] = model_all.predict(df_test[feats])
# df_sub = df_test[['id_encode','ds','pred']]
# df_sub['ds'] = df_sub['ds'].astype(int)
# df_sub.columns = ['id_encode','ds','power']

# save_path = f'../data/output/oofrmse{oof_score}_allpred.csv'
# df_sub.to_csv(save_path,index=False)
# print(f'save sub {save_path}')

In [ ]:
df_test = lgbmodel.predict(df_test)

df_sub = df_test[['id_encode','ds','pred']]
df_sub['ds'] = df_sub['ds'].astype(int)
df_sub.columns = ['id_encode','ds','power']

save_path = f'../data/output/{current_time_str}_oofrmse{oof_score}.csv'
df_sub.to_csv(save_path,index=False)
print(f'save sub {save_path}')
df_sub

* 为什么 power_std相关特征如此有效，能否挖掘更有意义的特征
* 几个业务特征为什么有效，如何衍生
* 空缺值多的训练数据的处理
* power为0可能是哪些原因导致的(目前看f3对其有影响，但不完全相关)
* id如何编码合适(全局TE会掉分)